In [ ]:
from keras.datasets import mnist

# Load the mnist data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalizing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
import numpy as np
from skimage import transform

nsamples = X_train.shape[0], X_test.shape[0]

# Input image dimensions
img_rows, img_cols = X_train.shape[1], X_train.shape[2]

# Building our newly redefined training/testing data. The new data comprises
# the original images but modified such that each digit class has been rotated 
# a different angle in 10 degrees increments

# Made new images bit larger to avoid rotation spilling out of original boundaries
skin = 12
X_train_redf = np.zeros((nsamples[0],2*img_rows-2*skin,2*img_cols-2*skin))
X_test_redf = np.zeros((nsamples[1],2*img_rows-2*skin,2*img_cols-2*skin))
X_train_org = np.zeros((nsamples[0],2*img_rows-2*skin,2*img_cols-2*skin))
X_test_org = np.zeros((nsamples[1],2*img_rows-2*skin,2*img_cols-2*skin))

for case, junk in enumerate(['train','test']):

    for isample in range(nsamples[case]):
         
         template = np.zeros((2*img_rows,2*img_cols))
         if(case == 0):
           digit = y_train[isample]
           template[img_rows/2:img_rows/2+img_rows,img_cols/2:img_cols/2+img_cols] = \
           X_train[isample]
         else:
           digit = y_test[isample]
           template[img_rows/2:img_rows/2+img_rows,img_cols/2:img_cols/2+img_cols] = \
           X_test[isample]
         r_angle = 10.0*(digit+1)
         template_r = transform.rotate(template,angle=r_angle)
         if (case == 0):
             X_train_redf[isample] = \
             template_r[skin:2*img_rows-skin,skin:2*img_rows-skin]
             X_train_org[isample] = \
             template[skin:2*img_rows-skin,skin:2*img_rows-skin]            
         else:
             X_test_redf[isample] = \
             template_r[skin:2*img_rows-skin,skin:2*img_rows-skin]
             X_test_org[isample] = \
             template[skin:2*img_rows-skin,skin:2*img_rows-skin]

In [ ]:
from scipy.misc import imsave, imshow, imread

# A little visualization test
image_transf = np.zeros((2*img_rows-2*skin,2*img_cols-2*skin, 3))
index = 8700
image_transf[:,:,0] = image_transf[:,:,1] = \
image_transf[:,:,2] = X_train_org[index]
print y_train[index]

imshow(image_transf)

In [ ]:
from keras.utils import np_utils
from keras import backend as K

nb_classes = 10

# Setting new training/test cases and preparing for model fitting
if K.image_dim_ordering() == 'th':
    print('theano ordering')
    X_train_new = \
    X_train_redf.reshape(nsamples[0],1,X_train_redf.shape[1],X_train_redf.shape[2])
    X_test_new = \
    X_test_redf.reshape(nsamples[1],1,X_test_redf.shape[1],X_test_redf.shape[2])
    input_shape = (1,X_train_redf.shape[1],X_train_redf.shape[2])
else:
    X_train_new = \
    X_train_redf.reshape(nsamples[0],X_train_redf.shape[1],X_train_redf.shape[2],1)
    X_test_new = \
    X_test_redf.reshape(X_nsamples[1],X_test_redf.shape[1],X_test_redf.shape[2],1)
    input_shape = (X_train_redf.shape[1],X_train_redf.shape[2],1)

print('X_train_new shape:', X_train_new.shape)
print(X_train_new.shape[0], 'train samples')
print(X_test_new.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
digits_pos_train = []
for idigit in range(nb_classes):
    digits_pos_train.append(np.argwhere(y_train == idigit))

# Visualize the different digit orientations in new training data
digits = np.zeros((X_train_new.shape[2], 10*X_train_new.shape[3], 3))

for idigit in range(nb_classes):
    digits[:,idigit*X_train_new.shape[3]:(idigit+1)*X_train_new.shape[3],0] = \
    digits[:,idigit*X_train_new.shape[3]:(idigit+1)*X_train_new.shape[3],1] = \
    digits[:,idigit*X_train_new.shape[3]:(idigit+1)*X_train_new.shape[3],2] = \
    X_train_new[digits_pos_train[idigit][0]]

imshow(digits)
imsave('mnist_train_mod.jpg',digits)

In [ ]:
#Building and training default Convnet. Architecture comes from the mnist exaple in the
# Keras distro

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D

# Random state
rseed = 1337
np.random.seed(rseed)
batch_size = 128
nb_epoch = 6

# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        activation='relu', name='conv1',
                        input_shape=input_shape))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       activation='relu', name='conv2'))
model.add(MaxPooling2D(pool_size=pool_size,name='pooling1'))
model.add(Dropout(0.25,name='dropout1'))
model.add(Flatten(name='flatten1'))
model.add(Dense(128,activation='relu', name='dense1'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(nb_classes,name='dense2'))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train_new, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test_new, Y_test))
score = model.evaluate(X_test_new, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model.save('mnist_cnn_theano.h5')

In [ ]:
from keras.models import load_model

model = load_model('mnist_cnn_theano.h5')

In [ ]:
# Little prediction test with default convnet
index = 1810
probs = model.predict(X_test_new[index].reshape((1, 1, input_shape[1], input_shape[2])))
print Y_test[index], np.argmax(probs)

In [ ]:
# Functions from Keras blog to run gradient ascent 

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def generate_image_from_grad(index_class,model,input_shape):
    nsteps = 100
    input_img = model.input

    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers[1:]])
    layer_output = layer_dict['dense2'].output
    loss = K.mean(layer_output[:,index_class])

    # we compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, input_img)[0]
    grads = normalize(grads)

    # this function returns the loss and grads given the input picture
    iterate = K.function([input_img,K.learning_phase()], [loss, grads])

    # step size for gradient ascent
    step = 1.0

    if K.image_dim_ordering() == 'th':
        tensor_shape = (1, 1, input_shape[1], input_shape[2])
    else:
        tensor_shape = (1, input_shape[1], input_shape[2], 1)

    # input image initialization
    input_img_data = np.random.random(tensor_shape)
    #input_img_data = np.ones(tensor_shape)
    
    # Gradient reconstruction loop
    for istep in range(nsteps):
        loss_value, grads_value = iterate([input_img_data,0])
        input_img_data += grads_value * step
        if istep % 10 == 0: print "Current loss value:", loss_value 
    
    # Prediction testing of reconstructed image
    probs = model.predict(input_img_data.reshape((tensor_shape)))
    print "Predicted class", np.argmax(probs)
     
    print "Done with class:", index_class
    return input_img_data
    

In [ ]:
# Generating the digits as seen by the convnet
digits_reconstructed = []
for idigit in range(nb_classes):
    input_img_data = generate_image_from_grad(idigit,model,input_shape)
    digits_reconstructed.append(input_img_data)

In [ ]:
# Function from Keras blog to turn tensor into image

def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    if K.image_dim_ordering() == 'th':
        x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# Visualizing/saving all reconstructed digits in one single image
# Note the incremental distinct rotations applied to each digit

border = 1
img_all_digits = \
np.zeros((X_train_new.shape[2]+2*border,nb_classes*(X_train_new.shape[3]+border)+border,3))

for idigit in range(nb_classes):
    img_all_digits[border:X_train_new.shape[2]+border,\
                   idigit*X_train_new.shape[3]+border+idigit:\
                   (idigit+1)*X_train_new.shape[3]+border+idigit,0] = \
    img_all_digits[border:X_train_new.shape[2]+border,\
                   idigit*X_train_new.shape[3]+border+idigit:\
                   (idigit+1)*X_train_new.shape[3]+border+idigit,1] = \
    img_all_digits[border:X_train_new.shape[2]+border,\
                   idigit*X_train_new.shape[3]+border+idigit:\
                   (idigit+1)*X_train_new.shape[3]+border+idigit,2] = \
    deprocess_image(digits_reconstructed[idigit][0])[:,:,0]

imshow(img_all_digits)
imsave('mnist_convnet_rot_digits.jpg',img_all_digits)

In [ ]:
# Setting weight matrix for Spatial Transformer network
# This is the STN implementation from Eder Santana (seya)
# that I have renamed

from Spatial_transformer import SpatialTransformerSeya

nparams = 6
nconnections = 50

b = np.zeros((2, 3), dtype='float32')
b[0, 0] = 1
b[1, 1] = 1
W = np.zeros((nconnections, nparams), dtype='float32')
weights = [W, b.flatten()]

In [ ]:
# Building localization net. Same as in the Seya example.

nfliters_loc = 20
locnet = Sequential()
locnet.add(Convolution2D(nfliters_loc,kernel_size[0],kernel_size[1],input_shape=input_shape))
locnet.add(MaxPooling2D(pool_size=pool_size))
locnet.add(Convolution2D(nfliters_loc, kernel_size[0], kernel_size[1]))
locnet.add(MaxPooling2D(pool_size=pool_size))

locnet.add(Flatten())
locnet.add(Dense(nconnections))
locnet.add(Dropout(0.5))
locnet.add(Activation('relu'))
locnet.add(Dense(nparams, weights=weights))

In [ ]:
# Building second convnet with transformer added. Uses same architecture as default
# convnet

# Random state
rseed = 1337
np.random.seed(rseed)
nb_epoch = 6

model_st = Sequential()

model_st.add(SpatialTransformerSeya(localization_net=locnet,
                             downsample_factor=1, input_shape=input_shape))
model_st.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1], 
                           activation='relu', name='conv1'))
model_st.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       activation='relu', name='conv2'))
model_st.add(MaxPooling2D(pool_size=pool_size,name='pooling1'))
model_st.add(Dropout(0.25,name='dropout1'))
model_st.add(Flatten(name='flatten1'))
model_st.add(Dense(128,activation='relu', name='dense1'))
model_st.add(Dropout(0.5,name='dropout2'))
model_st.add(Dense(nb_classes,name='dense2'))
model_st.add(Activation('softmax'))

model_st.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
# Preparing training data for transformer network. Includes original mnist 
# images (unrotated) as well as the rotated ones.

X_train_st = \
np.empty((2*X_train_new.shape[0],X_train_new.shape[1],\
          X_train_new.shape[2],X_train_new.shape[3]))
X_test_st = np.empty((2*X_test_new.shape[0],X_test_new.shape[1],\
                      X_test_new.shape[2],X_test_new.shape[3]))
Y_train_st = np.empty((2*Y_train.shape[0],Y_train.shape[1]))
Y_test_st = np.empty((2*Y_test.shape[0],Y_test.shape[1]))

X_train_st[0:X_train_new.shape[0]] = X_train_new
X_test_st[0:X_test_new.shape[0]] = X_test_new

Y_train_st[0:Y_train.shape[0]] = Y_train
Y_train_st[Y_train.shape[0]:2*Y_train.shape[0]] = Y_train
Y_test_st[0:Y_test.shape[0]] = Y_test
Y_test_st[Y_test.shape[0]:2*Y_test.shape[0]] = Y_test

if K.image_dim_ordering() == 'th':
    print('theano ordering'), X_train_new.shape[0]
    X_train_st[X_train_new.shape[0]:2*X_train_new.shape[0]] = \
    X_train_org.reshape(X_train_org.shape[0],1,X_train_org.shape[1],X_train_org.shape[2])
    X_test_st[X_test_new.shape[0]:2*X_test_new.shape[0]] = \
    X_test_org.reshape(X_test_org.shape[0],1,X_test_org.shape[1],X_test_org.shape[2])
else:
    X_train_st[X_train_new.shape[0]:2*X_train_new.shape[0]] = \
    X_train_org.reshape(X_train_org.shape[0],X_train_org.shape[1],X_train_org.shape[2],1)
    X_test_st[X_test_new.shape[0]:2*X_test_new.shape[0]] = \
    X_test_org.reshape(X_test_org.shape[0],X_test_org.shape[1],X_test_org.shape[2],1)
 
# Shuffling rotated and unrotated images in the training/testing sets

indices_train = np.random.permutation(range(2*X_train_new.shape[0]))
X_train_st = X_train_st[indices_train]
Y_train_st = Y_train_st[indices_train]

indices_test = np.random.permutation(range(2*X_test_new.shape[0]))
X_test_st = X_test_st[indices_test]
Y_test_st = Y_test_st[indices_test]

print('X_train_st shape:', X_train_st.shape)
print(X_train_st.shape[0], 'train samples')
print(X_test_st.shape[0], 'test samples')

In [ ]:
# A little visualization test
image_transf = np.zeros((2*img_rows-2*skin,2*img_cols-2*skin, 3))
index = 133
image_transf[:,:,0] = image_transf[:,:,1] = \
image_transf[:,:,2] = X_test_st[index]
print np.argmax(Y_test_st[index])

imshow(image_transf)

In [ ]:
# Training transformer data
model_st.fit(X_train_st, Y_train_st, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test_st, Y_test_st))
score = model.evaluate(X_test_st, Y_test_st, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Performing surgery on transformer convnet.
# Define new convnet identical to transformer one but without first localization net 
# layer. Import weights from trained ST convnet 

model_st_partial = Sequential()

model_st_partial.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        activation='relu', name='conv1',
                        input_shape=input_shape))
model_st_partial.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                       activation='relu', name='conv2'))
model_st_partial.add(MaxPooling2D(pool_size=pool_size,name='pooling1'))
model_st_partial.add(Dropout(0.25,name='dropout1'))
model_st_partial.add(Flatten(name='flatten1'))
model_st_partial.add(Dense(128,activation='relu', name='dense1'))
model_st_partial.add(Dropout(0.5,name='dropout2'))
model_st_partial.add(Dense(nb_classes,name='dense2'))
model_st_partial.add(Activation('softmax'))

for l_index in range(len(model_st.layers)-1):
    weights = model_st.layers[l_index+1].get_weights()
    model_st_partial.layers[l_index].set_weights(weights)

In [ ]:
model_st_partial.save('mnist_cnn_theano_st.h5')

In [ ]:
model_st_partial = load_model('mnist_cnn_theano_st.h5')

In [ ]:
# Digits as seen by convnet derived from transformer 

digits_reconstructed_partial = []
for idigit in range(nb_classes):
    input_img_data = generate_image_from_grad(idigit,model_st_partial,input_shape)
    digits_reconstructed_partial.append(input_img_data)

In [ ]:
# Visualizing all reconstructed digits in one single image
# Note what appears to be universal alignment and/or rotation-free
# reconstructed morphologies

border = 1
img_all_digits = \
np.zeros((X_train_st.shape[2]+2*border,nb_classes*(X_train_st.shape[3]+border)+border,3))

for idigit in range(nb_classes):
    img_all_digits[border:X_train_st.shape[2]+border,\
                   idigit*X_train_st.shape[3]+border+idigit:\
                   (idigit+1)*X_train_st.shape[3]+border+idigit,0] = \
    img_all_digits[border:X_train_st.shape[2]+border,\
                   idigit*X_train_st.shape[3]+border+idigit:\
                   (idigit+1)*X_train_st.shape[3]+border+idigit,1] = \
    img_all_digits[border:X_train_st.shape[2]+border,\
                   idigit*X_train_st.shape[3]+border+idigit:\
                   (idigit+1)*X_train_st.shape[3]+border+idigit,2] = \
    deprocess_image(digits_reconstructed_partial[idigit][0])[:,:,0]

imshow(img_all_digits)
imsave('mnist_convnet_st_digits.jpg',img_all_digits)

In [ ]:
# Little prediction test passing original unrotated data directly to
# convnet after transformer surgery

index = digits_pos_train[5][2000]
probs = \
model_st_partial.predict(X_train_new[index].reshape((1, 1, input_shape[1], input_shape[2])))
print Y_train[index], np.argmax(probs)